In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

#### Gauss Jacobi Iteration Method

$$
x_i^{k+1} = \left(b_i -\sum_{j=1}^{i-1} a_{ij}x_j^k -\sum_{j=i+1}^{n} a_{ij}x_j^k\right)/a_{ii}
$$

In a condensed form:
$$
x_i^{k+1} = \left(b_i -\sum_{j=1, j\neq i}^n a_{ij}x_j^k\right)/a_{ii}
$$


How do I define $x_i^{k+1} - x_i^k$?

Let $e_i = x_i^{k+1} - x_i^k$

1. Measure of convergence is max($e_i$) < Tolerance (maximum error between old and new values of x will be compared with a user-defined Tolerance) $L_{\infty}$ norm

2. $L_2$ norm: $\sqrt{\sum_{i=1}^N e_i^2}$

##### Consider set of linear equations

$$
10 x_1 - x_2 + 2 x_3 = 6,
$$

$$
-x_1 + 11 x_2 -x_3 + 3 x_4 = 25,
$$

$$
2 x_1 - x_2 + 10x_3 -x_4 = -11,
$$

$$
-3x_2 -x_3 +8x_4 = 15
$$

In [4]:
#create Ax = b, using numpy arrays

A = np.array([[10,-1,2,0],[-1,11,-1,3],[2,-1,10,-1],[0,-3,-1,8]])
b = np.array([[6,25,-11,15]])
b = np.transpose(b)

In [5]:
A

array([[10, -1,  2,  0],
       [-1, 11, -1,  3],
       [ 2, -1, 10, -1],
       [ 0, -3, -1,  8]])

In [6]:
b

array([[  6],
       [ 25],
       [-11],
       [ 15]])

In [7]:
np.diagonal(A)

array([10, 11, 10,  8])

In [8]:
D = np.diag(np.diagonal(A))
D

array([[10,  0,  0,  0],
       [ 0, 11,  0,  0],
       [ 0,  0, 10,  0],
       [ 0,  0,  0,  8]])

In [9]:
LU = A - np.diag(np.diagonal(A))
LU

array([[ 0, -1,  2,  0],
       [-1,  0, -1,  3],
       [ 2, -1,  0, -1],
       [ 0, -3, -1,  0]])

In [10]:
x_o = np.zeros_like(b)
x_o

array([[0],
       [0],
       [0],
       [0]])

In [11]:
x = np.zeros_like(b)

In [12]:
x[:] = (b - np.dot(LU,x_o))
x

array([[  6],
       [ 25],
       [-11],
       [ 15]])

In [13]:
Dinv = np.linalg.inv(D)
Dinv

array([[0.1       , 0.        , 0.        , 0.        ],
       [0.        , 0.09090909, 0.        , 0.        ],
       [0.        , 0.        , 0.1       , 0.        ],
       [0.        , 0.        , 0.        , 0.125     ]])

In [14]:
np.dot(Dinv, x)

array([[ 0.6       ],
       [ 2.27272727],
       [-1.1       ],
       [ 1.875     ]])

In [15]:
n = np.size(b)
n

4

In [16]:
np.diagonal(A)

array([10, 11, 10,  8])

In [17]:
z = np.reshape(np.diagonal(A), [n,1])
#z.shape
z

array([[10],
       [11],
       [10],
       [ 8]])

In [18]:
x

array([[  6],
       [ 25],
       [-11],
       [ 15]])

In [19]:
x/z

array([[ 0.6       ],
       [ 2.27272727],
       [-1.1       ],
       [ 1.875     ]])

In [20]:
## What if we have not reshaped z
z = np.array([[3, 3, 3, 3]])
z

x
x/z

array([[ 2.        ,  2.        ,  2.        ,  2.        ],
       [ 8.33333333,  8.33333333,  8.33333333,  8.33333333],
       [-3.66666667, -3.66666667, -3.66666667, -3.66666667],
       [ 5.        ,  5.        ,  5.        ,  5.        ]])

In [21]:
z = np.reshape(np.diagonal(A), [n,1])
#z.shape
z

array([[10],
       [11],
       [10],
       [ 8]])

In [22]:
x = (b - np.dot(LU,x_o))/z
x

array([[ 0.6       ],
       [ 2.27272727],
       [-1.1       ],
       [ 1.875     ]])

In [23]:
x_o

array([[0],
       [0],
       [0],
       [0]])

In [24]:
##error
e = np.linalg.norm(x-x_o)
e

3.201704898362488

In [25]:
e = np.linalg.norm(x-x_o, ord='fro')
e

3.201704898362488

In [26]:
### Putting all the above steps together in a iterative loop

import numpy as np
max_iter = 50
Tol = 1.e-9

A = np.array([[10,-1,2,0],[-1,11,-1,3],[2,-1,10,-1],[0,-3,-1,8]])
b = np.array([[6,25,-11,15]])
b = np.transpose(b)
n = np.size(b)
z = np.reshape(np.diagonal(A),[n,1])
LU = A - np.diag(np.diagonal(A))
x_o = np.zeros_like(b)
for i in range(max_iter):
    x = (b - np.dot(LU,x_o))/z
    e = np.linalg.norm(x-x_o, ord='fro')
    # print('after {} iteration:'.format(i))
    # print('x={}'.format(x))
    # print('error:{:.4e}\n'.format(e))
    if e < Tol:
        print(f'Iterations converged after {i} steps')
        break
    x_o = x

Iterations converged after 18 steps


In [27]:
### Putting it all together in a python function
def myJacobi(A,b,max_iter,Tolerance):
    n = np.size(A[0])
    b = np.reshape(b,[n,1])
    z = np.reshape(np.diagonal(A),[n,1])
    # i != j
    B = A - np.diag(np.diagonal(A))
    x_o = np.zeros_like(b)
    for i in range(max_iter):
        x = (b - np.dot(B,x_o))/z
        e = np.linalg.norm(x-x_o, ord='fro')
        print(x,e)
        if e < Tol:
            print(f'Iterations converged after {i} steps')
            break
        x_o = x
    return x

### Gauss Seidel method

$$
x_i^{k+1} = \left(b_i - \sum_{j=1}^{i-1} a_{ij}x_j^{k+1} - \sum_{j=i+1}^n a_{ij}x_j^k\right)/a_{ii}
$$

##### Homework: 
1. Can you write down a python function for the Gauss-Seidel method?
2. Solve the above set of equations using your Gauss-Seidel implementation, and check which method converges faster.